In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# url 설정
url = 'https://statiz.sporki.com/stats/?m=team&m2=pitching&m3=starting&so=&ob=&year=2024&sy=2024&ey=2024&te=&po=&lt=10100&reg=&pe=M3&ds=&de=&we=2&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=50&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2='

# 웹 요청
response = requests.get(url)
response.encoding = 'utf-8'
html_content = response.text

In [3]:
# BeautifulSoul 객체 생성
soup = BeautifulSoup(html_content, 'html.parser')
temp = soup.find_all('table')[0]
temp

<table>
<thead>
<tr>
<th rowspan="2">Rank</th>
<th rowspan="2">Season</th>
<th class="sorting" ob="" rowspan="2" so="s_year" style="width:5rem;max-width:5rem;">Year</th>
<th class="align_center sorting" ob="DESC" so="ERA">정렬▲</th>
<th class="sorting" flow="down" ob="DESC" rowspan="2" so="G" tooltip="Games, 출장">G</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="GS" tooltip="Games Started, 선발">GS</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="W" tooltip="Wins , 승">W</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="L" tooltip="Loses, 패">L</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="S" tooltip="Saves, 세이브">S</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="HD" tooltip="Hold, 홀드">HD</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="IP" tooltip="Inning, 이닝">IP</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="R" tooltip="Runs, 실점">R</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="ER"

In [4]:
table = soup.find_all('table')[1]  # 테이블 찾고 지정

In [41]:
# 테이블 헤더 추출
headers = []
for th in table.find_all('th'):
    headers.append(th.get_text())

headers

['Rank',
 'Team',
 'Year',
 '정렬▲',
 'G',
 'GS',
 'W',
 'L',
 'S',
 'HD',
 'IP',
 'R',
 'ER',
 'RS',
 'RS9',
 'TBF',
 'AB',
 'H',
 '2B',
 '3B',
 'HR',
 'BB',
 'IB',
 'HP',
 'SO',
 'SF',
 'NP',
 '비율',
 'ERA▲',
 'ERA',
 'WHIP',
 'AVG',
 'OBP',
 'OPS']

In [42]:
headers.pop(27)
headers.pop(28)

'ERA'

In [67]:
# 팀 기록 데이터

rows = []
expected_columns = len(headers)

for row in table.find_all('tr')[:]:  # 헤더 제외
    row_data = []
    for cell in row.find_all('td'):
        text = cell.get_text().strip()  # 공백 제거
        row_data.append(text)


    rows.append(row_data)

rows_df = pd.DataFrame(rows, columns=headers)
rows_df

,Rank,Team,Year,정렬▲,G,GS,W,L,S,HD,...,IB,HP,SO,SF,NP,ERA▲,WHIP,AVG,OBP,OPS
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1,한화,24,0.00,5,1,1,0,0,0,...,0,2,8,0,150,0.67,0.100,0.229,0.329,0.00
3,2,KIA,24,1.00,5,1,1,0,1,0,...,0,0,8,0,148,1.22,0.182,0.289,0.501,1.00
4,3,키움,24,10.10,4,1,0,1,0,0,...,0,2,11,0,191,2.50,0.314,0.478,1.049,10.10
5,4,롯데,24,2.25,4,1,0,1,0,0,...,0,0,7,0,115,1.13,0.214,0.290,0.647,2.25
6,5,LG,24,3.00,4,1,1,0,0,0,...,0,1,6,0,131,1.11,0.229,0.289,0.518,3.00
7,6,두산,24,4.00,6,1,1,0,1,2,...,0,0,14,1,170,1.56,0.289,0.333,0.649,4.00
8,7,삼성,24,4.15,4,1,0,1,0,1,...,0,0,2,1,116,1.50,0.296,0.394,0.913,4.15
9,8,NC,24,5.00,5,1,1,0,0,0,...,0,1,11,0,153,1.11,0.229,0.289,0.660,5.00


In [69]:
# CSV 파일로 저장
rows_df.to_csv('team_pitching_stats.csv', index=False, encoding='utf-8')

### 실험중

In [48]:
# DataFrame 생성
# headers가 열 이름으로, rows가 데이터 행으로 가정
data_dict = {header: [] for header in headers}

# 각 row의 데이터를 해당 header에 맞게 data_dict에 추가
for row in rows:
   if len(row) == len(headers):
    for i, header in enumerate(headers):
      data_dict[header].append(row[i])

    else:
        print(f"Warning: Row has {len(row)} columns, expected {len(headers)}. Skipping this row.")
        # Handle the inconsistent row (e.g., skip, fill missing values, etc.)

df = pd.DataFrame(data_dict)
df.head()

,Rank,Team,Year,정렬▲,G,GS,W,L,S,HD,...,IB,HP,SO,SF,NP,ERA▲,WHIP,AVG,OBP,OPS
0,1,한화,24,0.00,5,1,1,0,0,0,...,0,2,8,0,150,0.67,0.100,0.229,0.329,0.00
1,2,KIA,24,1.00,5,1,1,0,1,0,...,0,0,8,0,148,1.22,0.182,0.289,0.501,1.00
2,3,키움,24,10.10,4,1,0,1,0,0,...,0,2,11,0,191,2.50,0.314,0.478,1.049,10.10
3,4,롯데,24,2.25,4,1,0,1,0,0,...,0,0,7,0,115,1.13,0.214,0.290,0.647,2.25
4,5,LG,24,3.00,4,1,1,0,0,0,...,0,1,6,0,131,1.11,0.229,0.289,0.518,3.00
